### Lógica:
- Extrair a lista de CEPs a partir de um dataframe e remover registros duplicados
- Buscar no database se um registro de CEP com localidade já existe (consumir menos dados da API e otimizar processamento)
- Se o registro existir, pular para o próximo CEP
- Se o registro não existir, consultar a localidade utilizando API externa
- Salvar o registro no DB e continuar a busca, até finalizar.
- Realizar a união dos valores de localidade do DB com os CEPs das localidades do relatório do BDO
- Reorganizar ordem das colunas do relatório e eliminar colunas inválidas
- Aplicar filtros de acordo com as regras de negócio e salvar as saídas em arquivos csv.

### Este jupyter notebook é responsável por agregar as funções e executar o código de forma mais clara ao usuário. Apesar disso, melhorias são necessárias para torná-lo mais legível.

In [17]:
#Esta função é responsável por carregar o arquivo CSV extraído do BDO e filtrar todos os CEPs e eliminar duplicados.
from utils.csv_manager import csv_loader

#Esta é responsável pelo tratamento dos dados do banco. Dentro do pacote db_manager existem as funções cep_loader, responsável
#por verificar se um registro já existe no BD e se não existir, fazer a consulta da localidade a partir do CEP.
#A função join_db_with_csv é responsável por fazer o join da localidade presente no DB, na planilha do BDO. Para tanto
#o valor do CEP é utilizado como chave única.
from utils import db_manager

In [1]:
caminho_do_csv = '../bdo_csv/bdo_report_16-02-2021-16-55-00.csv'
ceps_csv = csv_loader(caminho_do_csv)

Salva o csv localmente. Será utilizado como entrada para buscar os CEPs

In [2]:
ceps_csv.to_csv('../data/ceps_sem_localidades.csv', sep=',', header=['CEP'], index=False)

### Data cleaning finished. Now, start to populate the db

Esta função cria o DB e/ou tabelas se não existirem. Depois, verifica se um CEP já existe na tabela, se não, busca na API externa.

In [3]:
db_manager.cep_loader()

SQLite version: ('3.31.1',)
Inserindo registro, CEP: 78053378, Localidade: Morada do Ouro
Inserindo registro, CEP: 78135301, Localidade: Canelas
Inserindo registro, CEP: 78110735, Localidade: 23 de Setembro
Inserindo registro, CEP: 78110738, Localidade: 23 de Setembro
Inserindo registro, CEP: 78135315, Localidade: Canelas
Inserindo registro, CEP: 78725140, Localidade: Jardim Primavera II
Inserindo registro, CEP: 78089610, Localidade: Residencial Coxipó
Inserindo registro, CEP: 78045220, Localidade: Cep nao encontrado
Inserindo registro, CEP: 78110760, Localidade: 23 de Setembro
Inserindo registro, CEP: 78135340, Localidade: Canelas
Inserindo registro, CEP: 78550400, Localidade: Jardim Primavera
Inserindo registro, CEP: 78053424, Localidade: Morada do Ouro
Inserindo registro, CEP: 78110768, Localidade: 23 de Setembro
Inserindo registro, CEP: 78089614, Localidade: Residencial Coxipó
Inserindo registro, CEP: 78053428, Localidade: Morada do Ouro
Inserindo registro, CEP: 78110773, Localidad

Inserindo registro, CEP: 78553381, Localidade: Comunidade Betel
Inserindo registro, CEP: 78135591, Localidade: Centro-Sul
Inserindo registro, CEP: 78135593, Localidade: Centro-Sul
Inserindo registro, CEP: 78053675, Localidade: Tancredo Neves
Inserindo registro, CEP: 78070060, Localidade: Jardim Petrópolis
Inserindo registro, CEP: 78053678, Localidade: Tancredo Neves
Inserindo registro, CEP: 78053680, Localidade: Tancredo Neves
Inserindo registro, CEP: 78750000, Localidade: Jardim das Paineiras
Inserindo registro, CEP: 78070068, Localidade: Jardim Petrópolis
Inserindo registro, CEP: 78070070, Localidade: Jardim Petrópolis
Inserindo registro, CEP: 78094652, Localidade: Parque Atalaia
Inserindo registro, CEP: 78070078, Localidade: Jardim Petrópolis
Inserindo registro, CEP: 78094654, Localidade: Parque Atalaia
Inserindo registro, CEP: 78094656, Localidade: Parque Atalaia
Inserindo registro, CEP: 78152000, Localidade: São Matheus
Inserindo registro, CEP: 78094658, Localidade: Parque Atalaia

Inserindo registro, CEP: 78717455, Localidade: Jardim Residencial Carlos Bezerra I
Inserindo registro, CEP: 78144016, Localidade: Petrópolis
Inserindo registro, CEP: 78070290, Localidade: Jardim Califórnia
Inserindo registro, CEP: 78144018, Localidade: Petrópolis
Inserindo registro, CEP: 78099481, Localidade: Jardim Industriário
Inserindo registro, CEP: 78750240, Localidade: Parque Residencial Universitário
Inserindo registro, CEP: 78144032, Localidade: Petrópolis
Inserindo registro, CEP: 78144034, Localidade: Petrópolis
Inserindo registro, CEP: 78553643, Localidade: Jardim dos Ipês
Inserindo registro, CEP: 78070316, Localidade: Jardim Califórnia
Inserindo registro, CEP: 78070320, Localidade: Jardim Califórnia
Inserindo registro, CEP: 78553649, Localidade: Jardim dos Ipês
Inserindo registro, CEP: 78553653, Localidade: Jardim dos Ipês
Inserindo registro, CEP: 78717500, Localidade: Jardim Residencial Carlos Bezerra II
Inserindo registro, CEP: 78717505, Localidade: Jardim Residencial Carl

Inserindo registro, CEP: 78750541, Localidade: Vila Rica
Inserindo registro, CEP: 78144334, Localidade: Petrópolis
Inserindo registro, CEP: 78734162, Localidade: Conjunto Habitacional Cidade de Deus
Inserindo registro, CEP: 78140254, Localidade: Glória
Inserindo registro, CEP: 78717812, Localidade: Loteamento Parque das Rosas
Inserindo registro, CEP: 78717814, Localidade: Loteamento Parque das Rosas
Inserindo registro, CEP: 78717818, Localidade: Loteamento Parque das Rosas
Inserindo registro, CEP: 78070650, Localidade: Praeirinho
Inserindo registro, CEP: 78717830, Localidade: Loteamento Parque das Rosas
Inserindo registro, CEP: 78140258, Localidade: Glória
Inserindo registro, CEP: 78144391, Localidade: Petrópolis
Inserindo registro, CEP: 78005130, Localidade: Centro-Norte
Inserindo registro, CEP: 78144394, Localidade: Petrópolis
Inserindo registro, CEP: 78144396, Localidade: Petrópolis
Inserindo registro, CEP: 78070670, Localidade: Praeirinho
Inserindo registro, CEP: 78140260, Localida

Inserindo registro, CEP: 78750776, Localidade: Jardim Tancredo Neves
Inserindo registro, CEP: 78095420, Localidade: Parque Cuiabá
Inserindo registro, CEP: 78750780, Localidade: Jardim Tancredo Neves
Inserindo registro, CEP: 78005310, Localidade: Centro-Norte
Inserindo registro, CEP: 78095422, Localidade: Parque Cuiabá
Inserindo registro, CEP: 78095423, Localidade: Parque Cuiabá
Inserindo registro, CEP: 78095425, Localidade: Parque Cuiabá
Inserindo registro, CEP: 78095426, Localidade: Parque Cuiabá
Inserindo registro, CEP: 78095427, Localidade: Parque Cuiabá
Inserindo registro, CEP: 78095428, Localidade: Parque Cuiabá
Inserindo registro, CEP: 78095429, Localidade: Parque Cuiabá
Inserindo registro, CEP: 78095430, Localidade: Parque Cuiabá
Inserindo registro, CEP: 78095424, Localidade: Parque Cuiabá
Inserindo registro, CEP: 78005320, Localidade: Centro-Norte
Inserindo registro, CEP: 78750788, Localidade: Jardim Tancredo Neves
Inserindo registro, CEP: 78750791, Localidade: Setor Rodoviária

...

Inserindo registro, CEP: 78093270, Localidade: São João Del Rei
Inserindo registro, CEP: 78060084, Localidade: Jardim Leblon
Inserindo registro, CEP: 78052312, Localidade: Altos da Serra I
Inserindo registro, CEP: 78091020, Localidade: Cep nao encontrado
Inserindo registro, CEP: 78740440, Localidade: Colina Verde
Inserindo registro, CEP: 78134236, Localidade: Capão do Pequi
Inserindo registro, CEP: 78060510, Localidade: Santo Antônio do Pedregal
Inserindo registro, CEP: 78740450, Localidade: Colina Verde
Inserindo registro, CEP: 78068708, Localidade: Boa Esperança
Inserindo registro, CEP: 78093285, Localidade: São João Del Rei
Inserindo registro, CEP: 78060520, Localidade: Santo Antônio do Pedregal
Inserindo registro, CEP: 78715881, Localidade: Residencial Padre Lothar
Inserindo registro, CEP: 78142442, Localidade: Mapim
Inserindo registro, CEP: 78715883, Localidade: Residencial Padre Lothar
Inserindo registro, CEP: 78150636, Localidade: Santa Isabel
Inserindo registro, CEP: 78715885, 

Inserindo registro, CEP: 78150788, Localidade: Santa Isabel
Inserindo registro, CEP: 78142596, Localidade: Mapim
Inserindo registro, CEP: 78552199, Localidade: Jardim das Violetas
Inserindo registro, CEP: 78050350, Localidade: Canjica
Inserindo registro, CEP: 78077065, Localidade: Santa Cruz
Inserindo registro, CEP: 78150792, Localidade: Santa Isabel
Inserindo registro, CEP: 78150791, Localidade: Santa Isabel
Inserindo registro, CEP: 78740620, Localidade: Vila Aurora
Inserindo registro, CEP: 78142604, Localidade: Mapim
Inserindo registro, CEP: 78716050, Localidade: Parque Residencial Buriti
Inserindo registro, CEP: 78142610, Localidade: Mapim
Inserindo registro, CEP: 78150802, Localidade: Santa Isabel
Inserindo registro, CEP: 78052502, Localidade: Morada da Serra
Inserindo registro, CEP: 78052504, Localidade: Morada da Serra
Inserindo registro, CEP: 78552217, Localidade: Jardim das Violetas
Inserindo registro, CEP: 78052506, Localidade: Morada da Serra
Inserindo registro, CEP: 78093470

Inserindo registro, CEP: 78052708, Localidade: Jardim Paraná
Inserindo registro, CEP: 78052710, Localidade: Jardim Paraná
Inserindo registro, CEP: 78093670, Localidade: Osmar Cabral
Inserindo registro, CEP: 78052712, Localidade: Jardim Paraná
Inserindo registro, CEP: 78115853, Localidade: Ponte Nova
Inserindo registro, CEP: 78052714, Localidade: Jardim Paraná
Inserindo registro, CEP: 78552427, Localidade: Jardim das Oliveiras
Inserindo registro, CEP: 78052716, Localidade: Jardim Paraná
Inserindo registro, CEP: 78052718, Localidade: Jardim Paraná
Inserindo registro, CEP: 78552430, Localidade: Jardim das Oliveiras
Inserindo registro, CEP: 78052720, Localidade: Jardim Paraná
Inserindo registro, CEP: 78052722, Localidade: Jardim Paraná
Inserindo registro, CEP: 78052724, Localidade: Jardim Paraná
Inserindo registro, CEP: 78028155, Localidade: Coophamil
Inserindo registro, CEP: 78110076, Localidade: Centro-Sul
Inserindo registro, CEP: 78552445, Localidade: Jardim das Oliveiras
Inserindo regi

Inserindo registro, CEP: 78552634, Localidade: Jardim das Azaléias
Inserindo registro, CEP: 78085700, Localidade: Vista Alegre
Inserindo registro, CEP: 78085703, Localidade: Vista Alegre
Inserindo registro, CEP: 78085710, Localidade: Vista Alegre
Inserindo registro, CEP: 78118480, Localidade: Cristo Rei
Inserindo registro, CEP: 78028370, Localidade: Novo Terceiro
Inserindo registro, CEP: 78118485, Localidade: Cristo Rei
Inserindo registro, CEP: 78138366, Localidade: Marajoara
Inserindo registro, CEP: 78028380, Localidade: Novo Terceiro
Inserindo registro, CEP: 78020190, Localidade: Centro-Sul
Inserindo registro, CEP: 78118504, Localidade: Cristo Rei
Inserindo registro, CEP: 78110325, Localidade: Centro-Norte
Inserindo registro, CEP: 78085750, Localidade: Vista Alegre
Inserindo registro, CEP: 78700150, Localidade: Centro
Inserindo registro, CEP: 78552695, Localidade: Jardim Maria Carolina
Inserindo registro, CEP: 78552698, Localidade: Jardim Maria Carolina
Inserindo registro, CEP: 78552

Inserindo registro, CEP: 78118830, Localidade: Cristo Rei
Inserindo registro, CEP: 78135216, Localidade: Canelas
Inserindo registro, CEP: 78700466, Localidade: Jardim Oliveira
Inserindo registro, CEP: 78118834, Localidade: Cristo Rei
Inserindo registro, CEP: 78143412, Localidade: Mapim
Inserindo registro, CEP: 78118840, Localidade: Cristo Rei
Inserindo registro, CEP: 78045120, Localidade: Quilombo
Inserindo registro, CEP: 78118852, Localidade: Cristo Rei
Inserindo registro, CEP: 78118855, Localidade: Cristo Rei
Inserindo registro, CEP: 78700491, Localidade: Vila Santo Antônio
Inserindo registro, CEP: 78143436, Localidade: Mapim
Inserindo registro, CEP: 78118859, Localidade: Cristo Rei
Inserindo registro, CEP: 78700498, Localidade: Vila Santo Antônio
Inserindo registro, CEP: 78553042, Localidade: Jardim Maria Vindilina II
Inserindo registro, CEP: 78045140, Localidade: Quilombo
Inserindo registro, CEP: 78553045, Localidade: Jardim Maria Vindilina II
Inserindo registro, CEP: 78053334, Loc

 - db_file -> Caminho para o db populado com as localidades. Por padrão, será criado na pasta '../data/cepsdb.db'
 - csv_file -> Caminho onde está salvo o relatório extraído do bdo, em formato csv.


In [5]:
db_file = '../data/cepsdb.db'
csv_file = '../bdo_csv/bdo_report_16-02-2021-16-55-00.csv'

df, nao_localizado = db_manager.join_db_with_csv(db_file, csv_file)

In [62]:
#df.head(5)

In [63]:
print(f'Os ceps não localizados são: {nao_localizado}\nSe estiver visualizando uma lista com valores iguais a "nan", \nsignifica que foram encontrados CEPs inválidos e por isso não foram associados.')

Os ceps não localizados são: ['nan', 'nan', 'nan']
Se estiver visualizando uma lista com valores iguais a "nan", 
significa que foram encontrados CEPs inválidos e por isso não foram associados.


# Filtrando dados de acordo com as regras de negócio

## Regras de negócio:

In [65]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.0f}'.format)

### Regra 1
* Retornar casos onde: $ CEP ~ Logradouro ~ CNEFE ⊂ CEP ~~ \& ~~ Alterar Logradouro para DNE? == NAO$

In [64]:
mask = dict()
for index, row in df.iterrows():
    #print(index)
    #print('logradouro CNEFE: ', int(row['CEP Logradouro CNEFE']))
    try:
        mask[index] = str(int(row['CEP Logradouro CNEFE'])) in row['CEP']
        #print(int(row['CEP Logradouro CNEFE']), row['CEP'])
        #print(int(row['CEP Logradouro CNEFE']) in row['CEP'])
        #print('\n\n')
    except:
        mask[index] = False

df['Filtro'] = pd.Series(mask)

#filtra por CEPs_CNEFE contidos na lista de CEPs DNE
df_cep_cnefe_in_cep_dne = df.loc[df['Filtro'] == True]
#Filtra pelo atributo Alterar Log... DNE == 'NAO'
df_alteraDNE_Nao = df_cep_cnefe_in_cep_dne.loc[df_cep_cnefe_in_cep_dne['Alterar Logradouro para DNE?'] == 'NAO']
df_alteraDNE_Nao.head(5)
#exporta dataframe em formato csv
df_alteraDNE_Nao.to_excel('../data/regra1_cepcnefe_in_cepdne_alteradne_nao.xlsx')

### Regra 2
* Retornar casos onde: $ CEP ~ Logradouro ~ CNEFE ⊄ CEP ~~ \& ~~ Alterar Logradouro para DNE? == SIM$

In [69]:
mask = dict()
for index, row in df.iterrows():
    #print(index)
    #print('logradouro CNEFE: ', int(row['CEP Logradouro CNEFE']))
    try:
        mask[index] = str(int(row['CEP Logradouro CNEFE'])) in row['CEP']
        #print(int(row['CEP Logradouro CNEFE']), row['CEP'])
        #print(int(row['CEP Logradouro CNEFE']) in row['CEP'])
        #print('\n\n')
    except:
        mask[index] = False

df['Filtro'] = pd.Series(mask)

#filtra por CEPs_CNEFE não contidos na lista de CEPs DNE
df_cep_cnefe_in_cep_dne = df.loc[df['Filtro'] == False]
#Filtra pelo atributo Alterar Log... DNE == 'NAO'
df_alteraDNE_Nao = df_cep_cnefe_in_cep_dne.loc[df_cep_cnefe_in_cep_dne['Alterar Logradouro para DNE?'] == 'SIM']
df_alteraDNE_Nao.head(5)
#exporta dataframe em formato csv
df_alteraDNE_Nao.to_excel('../data/regra2_cepcnefe_not_in_cepdne_alteradne_sim.xlsx')